In [16]:
library(tidyverse)


url<- "https://raw.githubusercontent.com/elaines6/dsci-100-group-project/main/processed.switzerland.data"
heart_raw_data <- read_csv(url, col_names = c("age", "sex", "chest_pain_type", "resting_bp", "chol", "fasting_bs", "rest_ecg", "max_rate",
                                             "exercise_induced_angina", "ST_depression", "slope_peak_exercise", "major_vessels", "thal", "diagnosis_heart_disease"))


heart_data_tidy <- heart_raw_data |>
    mutate(sex = as_factor(sex), chest_pain_type = as_factor(chest_pain_type), exercise_induced_angina = as_factor(exercise_induced_angina)) |>
    mutate(sex = fct_recode(sex, "Female" = "0", "Male" = "1")) |>
    mutate(exercise_induced_angina = fct_recode(exercise_induced_angina, "Yes" = "1", "No" = "0")) |>
    mutate(chest_pain_type = fct_recode(chest_pain_type, "Typical angina" = "1", "Atypical angina" = "2", "Non-anginal pain" = "3", "Asymptomatic" = "4")) |>

    select(age, sex, chest_pain_type, resting_bp, max_rate, exercise_induced_angina) 
    

heart_data_tidy


Rows: 123 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): resting_bp, fasting_bs, rest_ecg, max_rate, exercise_induced_angina...
dbl (5): age, sex, chest_pain_type, chol, diagnosis_heart_disease

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,chest_pain_type,resting_bp,max_rate,exercise_induced_angina
<dbl>,<fct>,<fct>,<chr>,<chr>,<fct>
32,Male,Typical angina,95,127,No
34,Male,Asymptomatic,115,154,No
35,Male,Asymptomatic,?,130,Yes
36,Male,Asymptomatic,110,125,Yes
38,Female,Asymptomatic,105,166,No
38,Female,Asymptomatic,110,156,No
38,Male,Non-anginal pain,100,179,No
38,Male,Non-anginal pain,115,128,Yes
38,Male,Asymptomatic,135,150,No
